# Battle of  Dubai Neighborhoods

# Code

Importing Panda, numpy,urlib.request and Beautiful Soup library

In [1]:
import pandas as pd
import numpy as np

In [2]:
# import the library we use to open URLs
import urllib.request

Getting XML output for required table in Wikipedia Page

In [3]:
# specify which URL/web page we are going to be scraping
url = "https://en.wikipedia.org/wiki/List_of_communities_in_Dubai"

In [4]:
# open the url using urllib.request and put the HTML into the page variable
page = urllib.request.urlopen(url)

In [5]:
# import the BeautifulSoup library so we can parse HTML and XML documents
from bs4 import BeautifulSoup

In [6]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup = BeautifulSoup(page, "lxml")

In [7]:
right_table=soup.find('table', class_='wikitable sortable')


Parsing XML to get data in lists against each header

In [8]:
a=[]
b=[]
c=[]
d=[]
e=[]
f=[]


for row in right_table.findAll('tr'):
    cells=row.findAll('td')
    if len(cells)==6:
            a.append(cells[0].find(text=True))
            b.append(cells[1].find(text=True))
            c.append(cells[2].find(text=True))
            d.append(cells[3].find(text=True))
            e.append(cells[4].find(text=True))
            f.append(cells[5].find(text=True))
       

Getting XML output for required table in www.propertymonitor.ae  Page and parsing it into Lists for each Header.

In [9]:
# specify which URL/web page we are going to be scraping
url1 = "https://www.propertymonitor.ae/research/uae-communities-index/dubai-rental-index.html"

In [10]:
# open the url using urllib.request and put the HTML into the page variable
page1 = urllib.request.urlopen(url1)

In [11]:
# parse the HTML from our URL into the BeautifulSoup parse tree format
soup1 = BeautifulSoup(page1, "lxml")

In [12]:
right_table1=soup1.find('table', class_='table table-bordered table-bordered-blue colBorderRight')


In [13]:
A=[]
B=[]
C=[]
D=[]
i = 0

for row in right_table1.findAll('tr'):
    
    cells=row.findAll('td')
    i = i+1
    if len(cells)==3:
        Hcells= row.findAll('th')
        if len(Hcells) == 1:
            A.append(Hcells[0].find(text=True))
            B.append(cells[0].find(text=True))
            C.append(cells[1].find(text=True))
            D.append(cells[2].find(text=True))
        else:
            
            A.append(A[i-3])
            B.append(cells[0].find(text=True))
            C.append(cells[1].find(text=True))
            D.append(cells[2].find(text=True))        

Inserting data froom wikipedia page into dubai_communities dataframe

In [14]:

dubai_communities=pd.DataFrame(b,columns=['Community'])
dubai_communities['Area']=d
dubai_communities['Population']=e


dubai_communities.head()


,Community,Area,Population
0,Abu Hail,1.27 km²,"21,414"
1,Al Awir First,None,None
2,Al Awir Second,None,None
3,Al Bada,0.82 km²,"18,816"
4,Al Baraha,1.104 km²,"7,823"


Inserting data froom  www.propertymonitor.ae Page  into dubai_rent dataframe

In [32]:
dubai_rent=pd.DataFrame(A,columns=['Community'])
dubai_rent['Type']=B
dubai_rent['Median Rent']=C
dubai_rent.head()

,Community,Type,Median Rent
0,Al Furjan,Apartment,"74,990"
1,Al Furjan,Villa,"134,421"
2,Al Habtoor City,Apartment,"95,922"
3,Al Khail Heights,Apartment,"43,110"
4,Arabian Ranches,Villa,"105,651"


Dropping all residence types apart from Apartment to have comparative Analysis

In [33]:
                                                                                  
dubai_rent.drop(dubai_rent[(dubai_rent['Type'].isin(['Villa','Garden Homes','Townhouse'])) ].index , inplace = True)
dubai_rent.head()

,Community,Type,Median Rent
0,Al Furjan,Apartment,"74,990"
2,Al Habtoor City,Apartment,"95,922"
3,Al Khail Heights,Apartment,"43,110"
6,Barsha Heights (Tecom),Apartment,"70,982"
7,Business Bay,Apartment,"87,078"


In [34]:
dubai_rent.reset_index(inplace = True)
dubai_rent.head()

,index,Community,Type,Median Rent
0,0,Al Furjan,Apartment,"74,990"
1,2,Al Habtoor City,Apartment,"95,922"
2,3,Al Khail Heights,Apartment,"43,110"
3,6,Barsha Heights (Tecom),Apartment,"70,982"
4,7,Business Bay,Apartment,"87,078"


In [35]:
dubai_rent.drop(columns = ['index'], inplace = True)

Installing geopy and importing other required librarires

In [20]:

!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge folium=0.5.0 --yes
import folium # plotting library

import geocoder

Solving environment: ...working... done

# All requested packages already installed.

Solving environment: ...working... done

# All requested packages already installed.



Defining Function to get Location Co-Ordinates for any address

In [22]:
def get_latilong(community):
    lati_long_coords = None
    
    while(lati_long_coords is None):
        
        g = geocoder.arcgis('{},Dubai,UAE'.format(community))
       
        lati_long_coords = g.latlng
        
    return lati_long_coords

Getting Co Ordinates for dubai communities and  joininng it into dubai_communities dataframe

In [23]:
# Retrieving Communities Co-ordinates
Community = dubai_communities['Community']    
coords = [ get_latilong(Community) for Community in Community.tolist() ]

In [24]:

# Adding Columns Latitude & Longitude
df_coords = pd.DataFrame(coords, columns=['Latitude', 'Longitude'])
dubai_communities['Latitude'] = df_coords['Latitude']
dubai_communities['Longitude'] = df_coords['Longitude']



In [25]:
dubai_communities.head()

,Community,Area,Population,Latitude,Longitude
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435
1,Al Awir First,None,None,25.18605,55.54108
2,Al Awir Second,None,None,25.16792,55.54331
3,Al Bada,0.82 km²,"18,816",25.21861,55.26406
4,Al Baraha,1.104 km²,"7,823",25.28280,55.31678


Getting Co Ordinates for communities in dubai_rent Dataframe

In [38]:
# Retrieving Communities Co-ordinates
Community = dubai_rent['Community']    
coords1 = [ get_latilong(Community) for Community in Community.tolist() ]

# Adding Columns Latitude & Longitude
df_coords1 = pd.DataFrame(coords1, columns=['Latitude', 'Longitude'])
dubai_rent['Latitude'] = df_coords1['Latitude']
dubai_rent['Longitude'] = df_coords1['Longitude']


In [39]:
dubai_rent.head()

,Community,Type,Median Rent,Latitude,Longitude
0,Al Furjan,Apartment,"74,990",25.276070,55.310870
1,Al Habtoor City,Apartment,"95,922",25.184240,55.254230
2,Al Khail Heights,Apartment,"43,110",25.182315,55.285566
3,Barsha Heights (Tecom),Apartment,"70,982",25.097230,55.171770
4,Business Bay,Apartment,"87,078",25.187970,55.262730


Identify neareast community in dubai_rent for community in dubai_community and add it to dataframe

In [28]:
dubai_communities['Nearby_Community'] = 'NA'
for ind in dubai_communities.index:
    distance_old = 10000
    distance_new = 10000
    for ind1 in dubai_rent.index:
        
        distance_new = np.sqrt((dubai_communities['Latitude'][ind] - dubai_rent['Latitude'][ind1]) ** 2 + (dubai_communities['Longitude'][ind] - dubai_rent['Longitude'][ind1]) ** 2)
        
        if distance_new < distance_old:
            distance_old = distance_new
            nearby_community = dubai_rent['Community'][ind1]
    
    dubai_communities['Nearby_Community'][ind] = nearby_community
    
   
    

C:\Users\Anupam\anaconda3\lib\site-packages\ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


In [29]:
dubai_communities

,Community,Area,Population,Latitude,Longitude,Nearby_Community
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,Al Furjan
1,Al Awir First,None,None,25.18605,55.54108,International City
2,Al Awir Second,None,None,25.16792,55.54331,International City
3,Al Bada,0.82 km²,"18,816",25.21861,55.26406,City Walk
4,Al Baraha,1.104 km²,"7,823",25.28280,55.31678,Al Furjan
...,...,...,...,...,...,...
126,Wadi Alamardi,None,None,25.21703,55.48636,International City
127,Warsan First,17.1 km²,"1,421",25.17685,55.41082,International City
128,Warsan Second,17.1 km²,"1,421",25.17025,55.44499,International City
129,Za'abeel First,10 km²,"5,283",25.22576,55.29937,Emirates Living


Create final Data Frame df_dxb by merging dubai_community and dubai_rent dataframes

In [40]:
dubai_rent.rename(columns = {'Community':'Nearby_Community', 'Latitude':'latitude_N', 'Longitude':'longitude_N'},inplace = True)

In [76]:
df_dxb = pd.merge( dubai_communities,dubai_rent,how = 'left', on='Nearby_Community')
df_dxb


,Community,Area,Population,Latitude,Longitude,Nearby_Community,Type,Median Rent,latitude_N,longitude_N
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,Al Furjan,Apartment,"74,990",25.27607,55.31087
1,Al Awir First,None,None,25.18605,55.54108,International City,Apartment,"25,162",25.17685,55.41082
2,Al Awir Second,None,None,25.16792,55.54331,International City,Apartment,"25,162",25.17685,55.41082
3,Al Bada,0.82 km²,"18,816",25.21861,55.26406,City Walk,Apartment,"151,369",25.20798,55.26185
4,Al Baraha,1.104 km²,"7,823",25.28280,55.31678,Al Furjan,Apartment,"74,990",25.27607,55.31087
...,...,...,...,...,...,...,...,...,...,...
126,Wadi Alamardi,None,None,25.21703,55.48636,International City,Apartment,"25,162",25.17685,55.41082
127,Warsan First,17.1 km²,"1,421",25.17685,55.41082,International City,Apartment,"25,162",25.17685,55.41082
128,Warsan Second,17.1 km²,"1,421",25.17025,55.44499,International City,Apartment,"25,162",25.17685,55.41082
129,Za'abeel First,10 km²,"5,283",25.22576,55.29937,Emirates Living,Apartment,"82,415",25.23625,55.26446


Generate Dubai Map with Circle marker for all neighborhoods

In [42]:
address = 'Dubai,UAE'
##address = 'dominos pizza discovery gardens, dubai'
geolocator = Nominatim(user_agent="foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

25.0750095 55.18876088183319


In [45]:
map_dubai = folium.Map(location=[latitude, longitude], zoom_start=10)

for lat, lng, nei in zip(df_dxb['Latitude'], df_dxb['Longitude'], df_dxb['Community']):
    
    label = '{}'.format(nei)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=3,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_dubai)  
    
map_dubai

Setting up Function to call Foursquare API for Venue Exploration. Output JSON file is coverted into Lists and then added to pandas dataframe

In [50]:
CLIENT_ID = 'SC10E2WBOHNRJ0PDH5A0J3OBNAQCN2MPG2FEH1QO2LLQVQPC' # your Foursquare ID
CLIENT_SECRET = 'STBG245KKKWJSI01ZVBWVTSW1122OW1DJ5ODMXBVUGMAG0P5' # your Foursquare Secret
VERSION = '20180605'
LIMIT = 30
print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: SC10E2WBOHNRJ0PDH5A0J3OBNAQCN2MPG2FEH1QO2LLQVQPC
CLIENT_SECRET:STBG245KKKWJSI01ZVBWVTSW1122OW1DJ5ODMXBVUGMAG0P5


In [51]:
import requests

In [52]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhoods', 
                  'Latitude', 
                  'Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

Get venue_details in dubai_venues dataframe using FourSquuare API Call

In [53]:

dubai_venues = getNearbyVenues(names=df_dxb['Community'],
                                   latitudes=df_dxb['Latitude'],
                                   longitudes=df_dxb['Longitude']
                                  )

Abu Hail
Al Awir First
Al Awir Second
Al Bada
Al Baraha
Al Barsha First
Al Barsha Second
Al Barsha South First
Al Barsha South Second
Al Barsha South Third
Al Barsha Third
Al Buteen
Al Dhagaya
Al Garhoud
Al Guoz Fourth
Al Hamriya, Dubai
Al Hamriya Port
Al Hudaiba
Al Jaddaf
Al Jafiliya
Al Karama
Al Khabisi
Al Khwaneej First
Al Khwaneej Second
Al Kifaf
Al Mamzar
Al Manara
Al Mankhool
Al Merkad
Al Mina
Al Mizhar First
Al Mizhar Second
Al Muraqqabat
Al Murar
Al Mushrif
Al Muteena
Al Nahda First
Al Nahda Second
Al Nasr, Dubai
Al Quoz First
Al Quoz Industrial First
Al Quoz Industrial Fourth
Al Quoz Industrial Second
Al Quoz Industrial Third
Al Quoz Second
Al Quoz Third
Al Qusais First
Al Qusais Industrial Fifth
Al Qusais Industrial First
Al Qusais Industrial Fourth
Al Qusais Industrial Second
Al Qusais Industrial Third
Al Qusais Second
Al Qusais Third
Al Raffa
Al Ras
Al Rashidiya
Al Rigga
Al Sabkha
Al Safa First
Al Safa Second
Al Safouh First
Al Safouh Second
Al Satwa
Al Shindagha
Al Souq Al

Analyzing dubai_venues dataframe

In [54]:
 dubai_venues.head()

,Neighborhoods,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Abu Hail,25.28308,55.33435,Habib Bakery,25.281124,55.332774,Bakery
1,Abu Hail,25.28308,55.33435,Gold's Gym,25.282698,55.341019,Gym
2,Abu Hail,25.28308,55.33435,Al Douri Roastery,25.277057,55.328223,Bakery
3,Abu Hail,25.28308,55.33435,Union Co-Operative Society,25.282769,55.340896,Department Store
4,Abu Hail,25.28308,55.33435,McDonald's,25.282839,55.340780,Fast Food Restaurant


In [55]:
dubai_venues.groupby('Neighborhoods').count()

,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhoods,,,,,,
Abu Hail,21,21,21,21,21,21
Al Awir First,4,4,4,4,4,4
Al Awir Second,5,5,5,5,5,5
Al Bada,30,30,30,30,30,30
Al Baraha,17,17,17,17,17,17
...,...,...,...,...,...,...
Wadi Alamardi,4,4,4,4,4,4
Warsan First,30,30,30,30,30,30
Warsan Second,9,9,9,9,9,9


In [56]:
dubai_venues.groupby('Venue Category').count()

,Neighborhoods,Latitude,Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
ATM,1,1,1,1,1,1
Accessories Store,3,3,3,3,3,3
Afghan Restaurant,8,8,8,8,8,8
African Restaurant,3,3,3,3,3,3
Airport Lounge,4,4,4,4,4,4
...,...,...,...,...,...,...
Women's Store,12,12,12,12,12,12
Yemeni Restaurant,1,1,1,1,1,1
Yoga Studio,2,2,2,2,2,2


In [57]:
print('There are {} uniques categories.'.format(len(dubai_venues['Venue Category'].unique())))

There are 270 uniques categories.


In [58]:
print('There are {} uniques Neighborhoods.'.format(len(dubai_venues['Neighborhoods'].unique())))

There are 131 uniques Neighborhoods.


Pivot the Venue Categories against neighborhood to get the total no of each venue category against a neighborhood

In [59]:
# one hot encoding
dubai_onehot = pd.get_dummies(dubai_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
dubai_onehot['Neighborhoods'] = dubai_venues['Neighborhoods'] 

# move neighborhood column to the first column
fixed_columns = [dubai_onehot.columns[-1]] + list(dubai_onehot.columns[:-1])
dubai_onehot = dubai_onehot[fixed_columns]



dubai_onehot.head()

,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,...,Vegetarian / Vegan Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Women's Store,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


The count is grouped by sum to get total availaibility of each Venue Categories. 

In [60]:
dubai_grouped = dubai_onehot.groupby('Neighborhoods').sum().reset_index()
print(dubai_grouped.shape)

dubai_grouped.head()

(131, 271)


,Neighborhoods,ATM,Accessories Store,Afghan Restaurant,African Restaurant,Airport Lounge,Airport Terminal,American Restaurant,Amphitheater,Antique Shop,...,Vegetarian / Vegan Restaurant,Warehouse Store,Water Park,Waterfront,Wine Bar,Women's Store,Yemeni Restaurant,Yoga Studio,Zoo,Zoo Exhibit
0,Abu Hail,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Al Awir First,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Al Awir Second,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Al Bada,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Al Baraha,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,1,0,0,0,0


Function definition to return sorted list of venues(count = num_top_venues) by it's total count . In case of any neighborhood having  venue types lesser than num_top_venues, it will return 0 for remaining items in List.

In [61]:

def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    i = 0
    for ind in row_categories_sorted.index:
        if row_categories_sorted.values[i] > 0:
            row_categories_sorted.values[i] = row_categories_sorted.index.values[i]
        i = i +1      
        
    return row_categories_sorted.values[0:num_top_venues]
 
        

Get top 10 venue types of each neighborhood into a new dataframe Neighborhoods_venues_sorted

In [62]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhoods']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
Neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
Neighborhoods_venues_sorted['Neighborhoods'] = dubai_grouped['Neighborhoods']

for ind in np.arange(dubai_grouped.shape[0]):
    Neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(dubai_grouped.iloc[ind, :], num_top_venues)
   

Neighborhoods_venues_sorted.head()

,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,Indian Restaurant,Department Store,Park,Fast Food Restaurant,Market,Bakery,Asian Restaurant,Convenience Store,Dessert Shop,Clothing Store
1,Al Awir First,Café,Cafeteria,0,0,0,0,0,0,0,0
2,Al Awir Second,Indian Restaurant,Coffee Shop,Burger Joint,Clothing Store,Café,0,0,0,0,0
3,Al Bada,Café,Coffee Shop,Middle Eastern Restaurant,Gym / Fitness Center,Ice Cream Shop,Supermarket,Beach,Bakery,Gluten-free Restaurant,Chinese Restaurant
4,Al Baraha,Hotel,Indian Restaurant,American Restaurant,Bar,Lounge,Asian Restaurant,Coffee Shop,Turkish Restaurant,Women's Store,Fried Chicken Joint


8 cluster of neighborhoods is created using K-Means Clustering by considering most common venues.

In [63]:
from sklearn.cluster import KMeans

In [64]:
# set number of clusters
kclusters = 8

dubai_grouped_clustering = dubai_grouped.drop('Neighborhoods', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(dubai_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 2, 0, 3, 3, 3, 2, 2, 2])

Add Cluster Level to dataframe

In [65]:
# add clustering labels
Neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)


In [68]:
print(Neighborhoods_venues_sorted.shape)
Neighborhoods_venues_sorted.head()

(131, 12)


,Cluster Labels,Neighborhoods,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,1,Abu Hail,Indian Restaurant,Department Store,Park,Fast Food Restaurant,Market,Bakery,Asian Restaurant,Convenience Store,Dessert Shop,Clothing Store
1,2,Al Awir First,Café,Cafeteria,0,0,0,0,0,0,0,0
2,2,Al Awir Second,Indian Restaurant,Coffee Shop,Burger Joint,Clothing Store,Café,0,0,0,0,0
3,0,Al Bada,Café,Coffee Shop,Middle Eastern Restaurant,Gym / Fitness Center,Ice Cream Shop,Supermarket,Beach,Bakery,Gluten-free Restaurant,Chinese Restaurant
4,3,Al Baraha,Hotel,Indian Restaurant,American Restaurant,Bar,Lounge,Asian Restaurant,Coffee Shop,Turkish Restaurant,Women's Store,Fried Chicken Joint


In [78]:
df_dxb_clustered = df_dxb.rename(columns ={'Community':'Neighborhoods'})
df_dxb_clustered.head(1)

,Neighborhoods,Area,Population,Latitude,Longitude,Nearby_Community,Type,Median Rent,latitude_N,longitude_N
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,Al Furjan,Apartment,"74,990",25.27607,55.31087


Create Dataframe by merging important informations from df_dxb dataframe

In [79]:
df_dxb_clustered = pd.merge( df_dxb_clustered,Neighborhoods_venues_sorted,how = 'left', on='Neighborhoods')

In [83]:
print(df_dxb_clustered.shape)
df_dxb_clustered.head(1)

(131, 21)


,Neighborhoods,Area,Population,Latitude,Longitude,Nearby_Community,Type,Median Rent,latitude_N,longitude_N,...,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,Al Furjan,Apartment,"74,990",25.27607,55.31087,...,Indian Restaurant,Department Store,Park,Fast Food Restaurant,Market,Bakery,Asian Restaurant,Convenience Store,Dessert Shop,Clothing Store


In [85]:
df_dxb_clustered.drop(columns = ['Nearby_Community','Type','latitude_N','longitude_N'], inplace = True)
df_dxb_clustered.head()

,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,"74,990",1,Indian Restaurant,Department Store,Park,Fast Food Restaurant,Market,Bakery,Asian Restaurant,Convenience Store,Dessert Shop,Clothing Store
1,Al Awir First,None,None,25.18605,55.54108,"25,162",2,Café,Cafeteria,0,0,0,0,0,0,0,0
2,Al Awir Second,None,None,25.16792,55.54331,"25,162",2,Indian Restaurant,Coffee Shop,Burger Joint,Clothing Store,Café,0,0,0,0,0
3,Al Bada,0.82 km²,"18,816",25.21861,55.26406,"151,369",0,Café,Coffee Shop,Middle Eastern Restaurant,Gym / Fitness Center,Ice Cream Shop,Supermarket,Beach,Bakery,Gluten-free Restaurant,Chinese Restaurant
4,Al Baraha,1.104 km²,"7,823",25.28280,55.31678,"74,990",3,Hotel,Indian Restaurant,American Restaurant,Bar,Lounge,Asian Restaurant,Coffee Shop,Turkish Restaurant,Women's Store,Fried Chicken Joint


Create Map with cluster and Rent Details. Each cluster will show in different colour

In [87]:
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors


In [90]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster,rent in zip(df_dxb_clustered['Latitude'], df_dxb_clustered['Longitude'], df_dxb_clustered['Neighborhoods'], df_dxb_clustered['Cluster Labels'],df_dxb_clustered['Median Rent']):
    label = folium.Popup(str(poi) + ': Cluster ' + str(cluster)+': Rent '+ str(rent), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [105]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 0].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 0]

(12, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
3,Al Bada,0.82 km²,"18,816",25.21861,55.26406,"151,369",0,Café,Coffee Shop,Middle Eastern Restaurant,Gym / Fitness Center,Ice Cream Shop,Supermarket,Beach,Bakery,Gluten-free Restaurant,Chinese Restaurant
22,Al Khwaneej First,None,None,25.24282,55.48440,"25,162",0,Café,Burger Joint,Coffee Shop,Comfort Food Restaurant,Juice Bar,Shopping Plaza,Bakery,Cafeteria,Poke Place,0
26,Al Manara,2.1 km²,"2,147",25.14579,55.20958,"128,948",0,Coffee Shop,Café,Farmers Market,Supermarket,Health & Beauty Service,Middle Eastern Restaurant,Australian Restaurant,Burger Joint,Spa,French Restaurant
61,Al Safouh First,7.6 km²,"3,337",25.11369,55.17547,"70,982",0,Café,Beach,College Gym,Gaming Cafe,Salon / Barbershop,BBQ Joint,Cafeteria,Grocery Store,Korean Restaurant,Gym / Fitness Center
88,Hor Al Anz East,4.18 km²,"40,342",25.27819,55.34710,"74,990",0,Middle Eastern Restaurant,Café,Smoke Shop,Department Store,Burrito Place,Burger Joint,Seafood Restaurant,Thai Restaurant,Gym,Fast Food Restaurant
93,Jumeira First,6.9 km,"26,499",25.21043,55.25260,"151,369",0,Café,Coffee Shop,Shopping Mall,Hotel,Burger Joint,Supermarket,Steakhouse,Perfume Shop,Middle Eastern Restaurant,Breakfast Spot
95,Jumeira Second,6.9 km,"26,499",25.19943,55.24683,"95,922",0,Café,Coffee Shop,Ice Cream Shop,Plaza,Tea Room,Salad Place,Steakhouse,Gift Shop,Lounge,Frozen Yogurt Shop
96,Jumeira Third,6.9 km,"26,499",25.18074,55.23161,"95,922",0,Café,Beach,Middle Eastern Restaurant,Ice Cream Shop,Resort,Restaurant,Bakery,Massage Studio,Grocery Store,Clothing Store
98,Mirdif,9.2 km²,"8,115",25.22614,55.42421,"25,162",0,Café,Coffee Shop,Supermarket,Middle Eastern Restaurant,Cosmetics Shop,Gym,Park,Restaurant,Bakery,Nail Salon
124,Umm Suqeim Second,7.2 km²,"16,459",25.15259,55.20044,"70,982",0,Café,Coffee Shop,Burger Joint,Beach,Theme Restaurant,Italian Restaurant,Juice Bar,Plaza,Poke Place,Restaurant


In [96]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 1].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 1].head()

(13, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Abu Hail,1.27 km²,"21,414",25.28308,55.33435,"74,990",1,Indian Restaurant,Department Store,Park,Fast Food Restaurant,Market,Bakery,Asian Restaurant,Convenience Store,Dessert Shop,Clothing Store
31,Al Mizhar Second,11.2 km²,"4,326",25.25236,55.45677,"25,162",1,Shopping Mall,Bakery,Indian Restaurant,Grocery Store,Coffee Shop,Park,Restaurant,Café,Sandwich Place,Burger Joint
39,Al Quoz First,27.1 km²,"16,719",25.16852,55.25065,"95,922",1,Coffee Shop,Furniture / Home Store,Spa,Mediterranean Restaurant,Fast Food Restaurant,Gym / Fitness Center,BBQ Joint,Shopping Mall,Bakery,Electronics Store
46,Al Qusais First,None,"15,502",25.28001,55.36860,"74,990",1,Fast Food Restaurant,Indian Restaurant,Currency Exchange,Fried Chicken Joint,Pizza Place,Soccer Stadium,Soccer Field,Burger Joint,Sandwich Place,Café
53,Al Qusais Third,None,"7,506",25.26064,55.39643,"124,464",1,Fast Food Restaurant,Park,Metro Station,Indian Restaurant,Burger Joint,Light Rail Station,Cafeteria,Basketball Court,Gas Station,Bagel Shop


In [97]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 2].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 2].head()

(50, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Al Awir First,None,None,25.18605,55.54108,"25,162",2,Café,Cafeteria,0,0,0,0,0,0,0,0
2,Al Awir Second,None,None,25.16792,55.54331,"25,162",2,Indian Restaurant,Coffee Shop,Burger Joint,Clothing Store,Café,0,0,0,0,0
7,Al Barsha South First,38.1 km²,1248,25.08958,55.23424,"61,118",2,Bakery,Fast Food Restaurant,Shopping Mall,Falafel Restaurant,Restaurant,Moving Target,Grocery Store,0,0,0
8,Al Barsha South Second,38.1 km²,1248,25.07739,55.24267,"61,118",2,Café,Supermarket,Hotel,Gym,Moving Target,Department Store,Coffee Shop,Office,Fast Food Restaurant,Shopping Mall
9,Al Barsha South Third,38.1 km²,1248,25.06229,55.23995,"60,305",2,Botanical Garden,Coffee Shop,Theme Park,Fast Food Restaurant,Burger Joint,Supermarket,Garden,Shopping Mall,Café,Cafeteria


In [98]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 3].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 3].head()

(17, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Al Baraha,1.104 km²,"7,823",25.28280,55.31678,"74,990",3,Hotel,Indian Restaurant,American Restaurant,Bar,Lounge,Asian Restaurant,Coffee Shop,Turkish Restaurant,Women's Store,Fried Chicken Joint
5,Al Barsha First,38.1 km²,1248,25.11483,55.19136,"70,982",3,Hotel,Middle Eastern Restaurant,Tapas Restaurant,Bed & Breakfast,Breakfast Spot,Korean Restaurant,Bakery,Steakhouse,Lebanese Restaurant,Russian Restaurant
6,Al Barsha Second,38.1 km²,1248,25.10723,55.20485,"70,982",3,Hotel,Coffee Shop,Indian Restaurant,Bakery,Ice Cream Shop,Park,Egyptian Restaurant,Gym / Fitness Center,Candy Store,Restaurant
13,Al Garhoud,4 km²,"4,466",25.24337,55.35267,"83,691",3,Airport Lounge,Coffee Shop,Seafood Restaurant,Hotel,Lounge,Duty-free Shop,Smoke Shop,Burger Joint,Chinese Restaurant,Pub
18,Al Jaddaf,7.3 km²,"2,990",25.22054,55.34166,"83,691",3,Hotel,French Restaurant,Buffet,Bar,Italian Restaurant,American Restaurant,Lounge,Middle Eastern Restaurant,Molecular Gastronomy Restaurant,Seafood Restaurant


In [99]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 4].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 4].head()

(17, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Al Hudaiba,0.84 km²,"7,699",25.23713,55.27707,"82,415",4,Coffee Shop,Asian Restaurant,Café,Convenience Store,Middle Eastern Restaurant,Indian Restaurant,Pizza Place,Seafood Restaurant,Cafeteria,Pakistani Restaurant
32,Al Muraqqabat,0.78 km²,"32,395",25.26836,55.32634,"74,990",4,Middle Eastern Restaurant,Hotel,Iraqi Restaurant,Gym / Fitness Center,Sandwich Place,Fried Chicken Joint,Spa,Café,Restaurant,Lounge
34,Al Mushrif,None,None,25.27257,55.37802,"124,464",4,Indian Restaurant,Restaurant,Fried Chicken Joint,Hotel,Coffee Shop,Café,Asian Restaurant,Bakery,Department Store,Dessert Shop
35,Al Muteena,1.12 km²,"18,094",25.27228,55.32291,"74,990",4,Coffee Shop,Hotel,Middle Eastern Restaurant,Café,Lounge,Iraqi Restaurant,Grocery Store,Nightclub,Shopping Mall,Fried Chicken Joint
36,Al Nahda First,3.18 km²,600,25.28660,55.36345,"74,990",4,Indian Restaurant,Department Store,Pakistani Restaurant,Middle Eastern Restaurant,Market,Beach,Residential Building (Apartment / Condo),Café,Business Service,Sandwich Place


In [100]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 5].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 5].head()

(7, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Al Buteen,0.07 km²,"2,364",25.26925,55.29944,"67,744",5,Café,Hotel,Market,Electronics Store,Historic Site,Art Gallery,Indian Restaurant,Asian Restaurant,Tea Room,Fast Food Restaurant
12,Al Dhagaya,0.125 km²,"10,896",25.27217,55.30157,"67,744",5,Hotel,Historic Site,Restaurant,Electronics Store,Art Gallery,Flower Shop,Shopping Mall,Café,Museum,Fast Food Restaurant
15,"Al Hamriya, Dubai",0.72 km²,"15,104",25.25696,55.30246,"67,744",5,Indian Restaurant,Tea Room,Café,Coffee Shop,Supermarket,Historic Site,Ice Cream Shop,Greek Restaurant,French Restaurant,Fast Food Restaurant
40,Al Quoz Industrial First,27.1 km²,"16,719",25.14525,55.23015,"128,948",5,Coffee Shop,Café,Gym / Fitness Center,Art Gallery,Cultural Center,Juice Bar,Soccer Field,Boutique,Middle Eastern Restaurant,Furniture / Home Store
55,Al Ras,0.20 km²,"6,812",25.26758,55.29459,"67,744",5,Indian Restaurant,Café,Historic Site,Coffee Shop,Museum,Art Gallery,Middle Eastern Restaurant,History Museum,Tea Room,Shopping Mall


In [101]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 6].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 6].head()

(8, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Al Karama,1.509 km,"45,674",25.24529,55.30364,"67,744",6,Indian Restaurant,Ice Cream Shop,North Indian Restaurant,Pakistani Restaurant,Filipino Restaurant,Fast Food Restaurant,Seafood Restaurant,Supermarket,Bakery,Restaurant
21,Al Khabisi,1.255 km²,"6,737",25.27177,55.33762,"74,990",6,Indian Restaurant,Hotel,Café,Chinese Restaurant,Market,Gym / Fitness Center,Convenience Store,Department Store,Pool Hall,Coffee Shop
24,Al Kifaf,0.8 km²,35,25.23809,55.29779,"67,744",6,Indian Restaurant,Hotel,Seafood Restaurant,Café,Park,Middle Eastern Restaurant,Pizza Place,Japanese Restaurant,Fast Food Restaurant,Tea Room
27,Al Mankhool,1 km,"16,013",25.24516,55.29333,"67,744",6,Indian Restaurant,Fast Food Restaurant,Supermarket,Chinese Restaurant,Restaurant,Hotel,Greek Restaurant,Dim Sum Restaurant,Seafood Restaurant,Burger Joint
48,Al Qusais Industrial First,None,"2,099",25.29176,55.38661,"124,464",6,Indian Restaurant,Ice Cream Shop,Café,Convenience Store,Restaurant,Office,Spa,Flower Shop,Fast Food Restaurant,Pakistani Restaurant


In [103]:
print(df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 7].shape)
df_dxb_clustered[df_dxb_clustered['Cluster Labels'] == 7].head()

(7, 17)


,Neighborhoods,Area,Population,Latitude,Longitude,Median Rent,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
14,Al Guoz Fourth,None,None,25.26951,55.30884,"67,744",7,Hotel,Asian Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Snack Place,Japanese Restaurant,Korean Restaurant,Lounge,Fried Chicken Joint,Fish Market
33,Al Murar,0.41 km²,"19,831",25.27835,55.30817,"74,990",7,Hotel,Indian Restaurant,Asian Restaurant,Middle Eastern Restaurant,Women's Store,Restaurant,Mobile Phone Shop,Fried Chicken Joint,Café,Skating Rink
38,"Al Nasr, Dubai",1.5 km²,"2,469",25.27604,55.30995,"74,990",7,Hotel,Asian Restaurant,Women's Store,Restaurant,Indian Restaurant,Japanese Restaurant,Middle Eastern Restaurant,Mobile Phone Shop,Skating Rink,Fast Food Restaurant
57,Al Rigga,0.89 km²,"5,684",25.26706,55.30890,"67,744",7,Hotel,Persian Restaurant,Restaurant,Middle Eastern Restaurant,Chinese Restaurant,Asian Restaurant,Korean Restaurant,Dim Sum Restaurant,Electronics Store,Snack Place
76,Ayal Nasir,0.17 km²,"13,077",25.27255,55.30313,"67,744",7,Hotel,Asian Restaurant,Art Gallery,Restaurant,Middle Eastern Restaurant,Japanese Restaurant,Jewelry Store,Gym,Fried Chicken Joint,Snack Place


In [106]:
df_dxb_clustered.to_csv(r'C:\Users\Anupam\Desktop\Lab\Course4\Battle_of_neighborhood.csv', index = False)